In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from pyspark.sql import *
#from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [8]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [9]:
sqlContext = SQLContext(sc)
geo_tags = sqlContext.read.csv('geo_tags.csv', header=True, inferSchema=True)
geo_tags_rdd = geo_tags.rdd

In [4]:
lenguajes = sqlContext.read.csv('languages.csv', header=True, inferSchema=True)
lenguajes_rdd = lenguajes.rdd

In [5]:
def organizacion_de_ususarios(lista):
    usuarios_que_entienden = 0
    usuarios_avanzados = 0
    ususarios_basicos = 0
    for x in lista:
        if x == "N" or x>"0":
            usuarios_que_entienden += 1
            if x>"3" or x == "N":
                usuarios_avanzados += 1
            else:
                ususarios_basicos += 1
    return [usuarios_que_entienden, usuarios_avanzados, ususarios_basicos]

In [6]:
rdd_usuarios_organizados=lenguajes_rdd.map(lambda x: (x[1], x[2])).groupByKey().map(lambda x:(x[0], organizacion_de_ususarios(list(x[1])))).map(lambda x: Row(lenguaje = x[0], usuarios_con_lectura_basica=x[1][0], usuarios_escritura_basica=x[1][2], usuarios_escritura_avanzada=x[1][1]))

In [7]:
spark.createDataFrame(rdd_usuarios_organizados).show()

+--------+---------------------------+-------------------------+---------------------------+
|lenguaje|usuarios_con_lectura_basica|usuarios_escritura_basica|usuarios_escritura_avanzada|
+--------+---------------------------+-------------------------+---------------------------+
|      FR|                          1|                        1|                          0|
|      aa|                          0|                        0|                          0|
|      ab|                          1|                        1|                          0|
|     ace|                          0|                        0|                          0|
|     acf|                          1|                        1|                          0|
|      af|                         10|                        9|                          1|
|     agr|                          1|                        1|                          0|
|     akk|                          1|                        1|      